In [11]:
!pip install -q -q -q geopandas
!pip install -q -q -q rtree

In [1]:
import geopandas as gpd
import shutil

In [2]:
shutil.unpack_archive('paris.zip')

Chargement des données carroyées et des quartiers de la politique de ville sur Paris

In [21]:
carreaux = gpd.read_file('data/paris.shp')

In [22]:
carreaux = carreaux[["Ind","Men","Men_pauv","Men_1ind","Men_5ind","Men_prop","Men_fmp","Ind_snv",
                   "Men_surf","Men_coll","Men_mais","Log_av45","Log_45_70","Log_70_90","Log_ap90",
                   "Log_inc","Log_soc","Ind_0_3","Ind_4_5","Ind_6_10","Ind_11_17","Ind_18_24",
                   "Ind_25_39","Ind_40_54","Ind_55_64","Ind_65_79","Ind_80p","Ind_inc", "geometry"]]

In [23]:
qpv = gpd.read_file('data/qp_bdtopo_com_75056_2020.shp')

Intersection spatiale entre les deux données spatiales

In [24]:
intersection = gpd.sjoin(qpv, carreaux, how='inner', op='intersects')

In [25]:
intersection.head(3)

,code,libelle,geometry,index_right,Ind,Men,Men_pauv,Men_1ind,Men_5ind,Men_prop,...,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",133,8.0,3.7,0.9,1.6,0.3,1.8,...,0.1,0.4,0.4,0.5,2.2,1.7,0.8,0.8,0.3,0.2
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",559,520.0,194.0,34.0,59.0,28.0,0.0,...,13.0,39.5,56.0,39.0,95.0,97.0,62.0,53.0,36.0,8.0
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",643,244.0,120.0,69.0,81.0,14.0,5.0,...,8.0,22.0,20.0,42.0,77.0,35.0,12.0,8.0,7.0,0.0


Aggrégation des données par qpv

In [26]:
qpv_statistics = intersection.dissolve(by=['code', 'libelle'], aggfunc='sum', as_index=False)

In [27]:
qpv_statistics.head(3)

,code,libelle,geometry,index_right,Ind,Men,Men_pauv,Men_1ind,Men_5ind,Men_prop,...,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc
0,QP075001,B?dier - Boutro,"POLYGON ((653957.500 6857921.700, 653802.699 6...",34133,5112.0,2150.7,444.9,889.6,237.3,141.8,...,175.6,365.4,498.9,294.0,1175.2,1126.7,494.8,394.8,173.3,96.2
1,QP075002,Chaufourniers,"POLYGON ((653876.162 6864327.010, 653913.377 6...",6625,10886.5,5048.0,955.0,2335.0,385.0,1482.0,...,242.0,570.0,822.0,545.5,2693.0,2265.0,1291.0,1276.0,445.0,257.5
2,QP075003,Villa d'Este,"POLYGON ((653491.700 6858200.401, 653559.901 6...",1794,4225.5,1505.0,378.0,417.0,251.0,403.0,...,128.0,284.5,426.0,266.5,895.0,953.0,461.0,362.0,138.0,74.0
